In [6]:
import capytaine as cpt
from capytaine.bem.airy_waves import airy_waves_free_surface_elevation
from capytaine.ui.vtk.animation import Animation
from capytaine.io.meshio import load_from_meshio

import wecopttool as wot


In [ ]:

cpt.set_logging('INFO')

# Generate the mesh of a sphere

full_mesh = cpt.mesh_fb(radius=3, center=(0, 0, 0), resolution=(20, 20))
full_sphere = cpt.FloatingBody(mesh=full_mesh)
full_sphere.add_translation_dof(name="Heave")

# Keep only the immersed part of the mesh
sphere = full_sphere.immersed_part()

# Set up and solve problem
solver = cpt.BEMSolver()

diffraction_problem = cpt.DiffractionProblem(body=sphere, wave_direction=0.0, omega=2.0)
diffraction_result = solver.solve(diffraction_problem)

radiation_problem = cpt.RadiationProblem(body=sphere, radiating_dof="Heave", omega=2.0)
radiation_result = solver.solve(radiation_problem)

# Define a mesh of the free surface and compute the free surface elevation
free_surface = cpt.FreeSurface(x_range=(-50, 50), y_range=(-50, 50), nx=150, ny=150)
diffraction_elevation_at_faces = solver.compute_free_surface_elevation(free_surface, diffraction_result)
radiation_elevation_at_faces = solver.compute_free_surface_elevation(free_surface, radiation_result)

# Add incoming waves
diffraction_elevation_at_faces = diffraction_elevation_at_faces + airy_waves_free_surface_elevation(free_surface, diffraction_problem)

# Run the animations
animation = Animation(loop_duration=diffraction_result.period)
animation.add_body(full_sphere, faces_motion=None)
animation.add_free_surface(free_surface, faces_elevation=0.5*diffraction_elevation_at_faces)
animation.run(camera_position=(-30, -30, 30))  # The camera is oriented towards (0, 0, 0) by default.
# animation.save("path/to/the/video/file.ogv", camera_position=(-30, -30, 30))

animation = Animation(loop_duration=radiation_result.period)
animation.add_body(full_sphere, faces_motion=full_sphere.dofs["Heave"])
animation.add_free_surface(free_surface, faces_elevation=3.0*radiation_elevation_at_faces)
animation.run(camera_position=(-30, -30, 30))
# animation.save("path/to/the/video/file.ogv", camera_position=(-30, -30, 30))

In [11]:
wb = wot.geom.WaveBot()  # use standard dimensions
mesh_size_factor = 0.5 # 1.0 for default, smaller to refine mesh
mesh = wb.mesh(mesh_size_factor)

# create mesh object for WaveBot and add internal lid
mesh_obj = load_from_meshio(mesh, 'WaveBot')
lid_mesh = mesh_obj.generate_lid(-2e-2)
full_fb = cpt.FloatingBody(mesh=mesh_obj, lid_mesh=lid_mesh, name="WaveBot")
full_fb.add_translation_dof(name="Heave")

fb = full_fb.immersed_part()



INFO:capytaine.io.meshio:Stored 1042 triangle faces as quadrilaterals


[16:34:57] INFO     Stored 1042 triangle faces as quadrilaterals

INFO:capytaine.bodies.bodies:New floating body: FloatingBody(mesh=Mesh(..., name="WaveBot"), lid_mesh=Mesh(..., name="lid for WaveBot"), dofs={}, name="WaveBot").


[16:34:58] INFO     New floating body: FloatingBody(mesh=Mesh(..., name="WaveBot"), lid_mesh=Mesh(..., name="lid   
                    for WaveBot"), dofs={}, name="WaveBot").

INFO:capytaine.bodies.bodies:Clipping WaveBot with respect to Plane(normal=[0. 0. 1.], point=[0. 0. 0.])


           INFO     Clipping WaveBot with respect to Plane(normal=[0. 0. 1.], point=[0. 0. 0.])

INFO:capytaine.meshes.clipper:Clipping lid for WaveBot by Plane(normal=[0. 0. 1.], point=[0. 0. 0.]): no action.


           INFO     Clipping lid for WaveBot by Plane(normal=[0. 0. 1.], point=[0. 0. 0.]): no action.

In [12]:
diffraction_problem = cpt.DiffractionProblem(body=fb, wave_direction=0.0, omega=2.0)
diffraction_result = solver.solve(diffraction_problem)

radiation_problem = cpt.RadiationProblem(body=fb, radiating_dof="Heave", omega=2.0)
radiation_result = solver.solve(radiation_problem)

# Define a mesh of the free surface and compute the free surface elevation
free_surface = cpt.FreeSurface(x_range=(-50, 50), y_range=(-50, 50), nx=150, ny=150)
diffraction_elevation_at_faces = solver.compute_free_surface_elevation(free_surface, diffraction_result)
radiation_elevation_at_faces = solver.compute_free_surface_elevation(free_surface, radiation_result)

# Add incoming waves
diffraction_elevation_at_faces = diffraction_elevation_at_faces + airy_waves_free_surface_elevation(free_surface, diffraction_problem)

# Run the animations
animation = Animation(loop_duration=diffraction_result.period)
animation.add_body(full_fb, faces_motion=None)
animation.add_free_surface(free_surface, faces_elevation=0.5*diffraction_elevation_at_faces)
animation.run(camera_position=(-30, -30, 30))  # The camera is oriented towards (0, 0, 0) by default.
# animation.save("path/to/the/video/file.ogv", camera_position=(-30, -30, 30))

animation = Animation(loop_duration=radiation_result.period)
animation.add_body(full_fb, faces_motion=full_fb.dofs["Heave"])
animation.add_free_surface(free_surface, faces_elevation=3.0*radiation_elevation_at_faces)
animation.run(camera_position=(-30, -30, 30))

INFO:capytaine.bem.solver:Solve DiffractionProblem(body=FloatingBody(..., name="WaveBot"), omega=2.000, water_depth=inf, wave_direction=0.000).


           INFO     Solve DiffractionProblem(body=FloatingBody(..., name="WaveBot"), omega=2.000, water_depth=inf, 
                    wave_direction=0.000).

INFO:capytaine.bem.solver:Solve RadiationProblem(body=FloatingBody(..., name="WaveBot"), omega=2.000, water_depth=inf, radiating_dof='Heave').


           INFO     Solve RadiationProblem(body=FloatingBody(..., name="WaveBot"), omega=2.000, water_depth=inf,   
                    radiating_dof='Heave').

INFO:capytaine.ui.vtk.animation:Precompute motions of free_surface_56_mesh before animation.


[16:36:30] INFO     Precompute motions of free_surface_56_mesh before animation.

INFO:capytaine.ui.vtk.animation:Precompute motions of WaveBot before animation.


[16:36:58] INFO     Precompute motions of WaveBot before animation.

INFO:capytaine.ui.vtk.animation:Precompute motions of free_surface_56_mesh before animation.


[16:37:02] INFO     Precompute motions of free_surface_56_mesh before animation.

In [13]:
#can I combine diffraction and radiation?

animation = Animation(loop_duration=radiation_result.period)
animation.add_body(full_fb, faces_motion=full_fb.dofs["Heave"])
animation.add_free_surface(free_surface, faces_elevation=radiation_elevation_at_faces+diffraction_elevation_at_faces)
animation.run(camera_position=(-30, -30, 30))

INFO:capytaine.ui.vtk.animation:Precompute motions of WaveBot before animation.


[16:37:33] INFO     Precompute motions of WaveBot before animation.

INFO:capytaine.ui.vtk.animation:Precompute motions of free_surface_56_mesh before animation.


[16:37:37] INFO     Precompute motions of free_surface_56_mesh before animation.